In [6]:
from lxml import html
import requests
import re
import pandas as pd
from datetime import datetime, timedelta
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [29]:
url = "https://www.congress.gov/congressional-record/2024/8/1/daily-digest"
print(url)

chamber = "Senate"

#     #function to get date string from congress.gov url   
#     def get_strg(url,chamber):


## ADDING IN SELENIUM

##set up selenium scraper
options = Options()
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

#since there is no more chromedriver.exe, Selenium should auto-detect the integrated driver
driver = webdriver.Chrome(options=options)
driver.get(url)
time.sleep(5)
# Parse the HTML content with lxml
page_content = driver.page_source
tree = html.fromstring(page_content)

driver.quit()

## ADDING IN SELENIUM

tree = html.fromstring(page_content) 
strg = clean(",".join(tree.xpath("//div[contains(text(),'Adjournment:')]")))
print(strg)
strg = strg.replace("Adjournment:","")
str_chamber = strg[:strg.find("House")] if chamber == "Senate" else strg[strg.find("House"):]

if chamber =="House of Representatives":
    chamber = "House"
    
if chamber not in strg:
    strg = clean(",".join(tree.xpath(f"//div[@class='main-wrapper']//center[contains(./h2/text(),'{chamber}')]//following-sibling::p[contains(./strong,'Adjournment:')]//text()")))
    print(strg)
    strg = (chamber +" "+ strg) if strg else None
    if not strg:
        strg = clean(",".join(tree.xpath(f"//div[@class='main-wrapper']//center[contains(./h2/text(),'{chamber}')]//following-sibling::p[contains(./strong,'Recess:')]//text()")))
        strg = (chamber +" "+ strg) if strg else None
        if not strg:
            strg = clean(str(tree.xpath(f"//div[@class='main-wrapper']//center[./h2/text()='{chamber}']//following-sibling::p[1]/text()")))
    str_chamber = strg

if not strg or not str_chamber:
    str_chamber = tree.xpath(f"//div[@class='main-wrapper']//center[./h2/text()='{chamber}']//following-sibling::p[1]/text()")
    print(str_chamber)
    str_chamber = [i for i in str_chamber if chamber in i]
    str_chamber = clean(str(str_chamber))
    if not str_chamber:
        str_chamber = tree.xpath("//div[@class='main-wrapper']//center[contains(./strong,'Adjournment:')]//following-sibling::text()[1]")
        str_chamber = [i for i in str_chamber if chamber in i]
        str_chamber = clean(str(str_chamber))

if not str_chamber:
    hld = tree.xpath(f"//div[@class='main-wrapper']/center[./h3/em/text()='Chamber Action']/preceding-sibling::center[h2[contains(./text(),'{chamber}')]]/following-sibling::p[contains(./text(),'{chamber}')][1]/text()")
    print(hld)
    hld = hld[0] if len(hld) > 0 else hld
    str_chamber = clean(str(hld))
    if not str_chamber:
        try:
            if chamber == "House":
                l = tree.xpath("//h3[./a[@name='daily-digest-house-of-representatives']]//following-sibling::div[1]//text()")
                print(l)
                str_chamber = clean(",".join([i for i in l if "Adjournment:" in i or "Chamber Action" in i]))
            else:
                l = tree.xpath("//h3[./a[@name='daily-digest-highlights-senate']]//following-sibling::div[1]//text()")
                print(l)
                vals = [i for i in range(len(l)) if "Adjournment:" in l[i] or "Chamber Action" in l[i]]
                new = [i+1 for i in vals]
                new2 = [i+2 for i in vals]
                vals = vals + new + new2

                str_chamber = clean(",".join([l[i] for i in vals]))
        except:
            try:
                if chamber == "House":
                    l = tree.xpath("//h3[./a[@name='daily-digest-house-of-representatives']]//following-sibling::div[1]//text()")
                    print(l)
                    str_chamber = clean(",".join([i for i in l if "Recess:" in i or "Chamber Action" in i]))
                else:
                    l = tree.xpath("//h3[./a[@name='daily-digest-highlights-senate']]//following-sibling::div[1]//text()")
                    print(l)
                    vals = [i for i in range(len(l)) if "Recess:" in l[i] or "Chamber Action" in l[i]]
                    new = [i+1 for i in vals]
                    new2 = [i+2 for i in vals]
                    vals = vals + new + new2

                    str_chamber = clean(",".join([l[i] for i in vals]))

            except:
                str_chamber = None
                
    if not str_chamber and chamber == "Senate":
        try:
            l = tree.xpath("//h3[./a[@name='daily-digest-senate']]//following-sibling::div[1]//text()")
            print(l)
            vals = [i for i in range(len(l)) if "Recess:" in l[i] or "Chamber Action" in l[i]]
            new = [i+1 for i in vals]
            new2 = [i+2 for i in vals]
            vals = vals + new + new2

            str_chamber = clean(",".join([l[i] for i in vals]))

        except:
            try:
                l = tree.xpath("//h3[./a[@name='daily-digest-senate']]//following-sibling::div[1]//text()")
                print(l)
                vals = [i for i in range(len(l)) if "Recess:" in l[i] or "Chamber Action" in l[i]]
                new = [i+1 for i in vals]
                new2 = [i+2 for i in vals]
                vals = vals + new + new2

                str_chamber = clean(",".join([l[i] for i in vals]))
            except:
                str_chamber = None
                

# if chamber not in str_chamber:
#     str_chamber = None

else:
    str_chamber = str_chamber.replace("at noon","12:00 pm")
    str_chamber = str_chamber.replace("noon","p.m.")
    if "12 midnight" in str_chamber:
        str_chamber = str_chamber.replace("12 midnight","12:00 am")
    elif "midnight" in str_chamber:
        str_chamber = str_chamber.replace("midnight","12:00 am")

https://www.congress.gov/congressional-record/2024/8/1/daily-digest


[]
[]
[]
[]


In [30]:
str_chamber

''

In [9]:
def clean(string):
    import re
    string = string.replace("\\xa0","")
    string = string.replace("[","")
    string = string.replace("]","")
    string = string.replace("'","")
    string = string.replace("\"","")
    string = string.replace("\\n","")
    string = string.replace("Sept","Sep")
    string = string.replace("July","Jul")
    string = string.replace("June","Jun")
    string = re.sub('\s+',' ',string)
    string = string.strip()
    return(string)

#function to format date strings into dates
def timeconv(x):
    x = re.sub("a.m.","AM",x)
    x = re.sub("p.m.","PM",x)
    x = re.sub("noon","PM",x)
    x = re.sub("\[","",x)
    x = re.sub("\]","",x)
    x = re.sub("\'","",x)
    x = re.sub(",","",x)
    try:
        val = datetime.strptime(x,"%I %p")
        return(val)

    except:
        try:
            val = datetime.strptime(x,"%I:%M %p")
            return(val)
        except:
            x = x.replace(".","")
            val = datetime.strptime(x,"%I:%M %p")
            return(val)

def workingdays(chamber, start, end):

    #format date
    if end == None:
        dateend = end = datetime.now().date()       
    else:
        dateend = datetime.strptime(end, '%m/%d/%Y').date()
    datestart = datetime.strptime(start, '%m/%d/%Y').date()
    #get date range
    diff = (dateend - datestart).days
    #set up results dataframe
    cols = ["House or Senate","Date","Time Convened","Time Adjourned","Working Day?","Link"]
    df = pd.DataFrame(columns=cols)

    #class/function to test if the url is functional and return next valid date and url
    class TestClass:
        def __init__(self,x,y):
            self.datestart = x
            self.url = y

    def web_test(datestart):

        url = "https://www.congress.gov/congressional-record/"+str('{d.year}/{d.month}/{d.day}'.format(d=datestart))+"/daily-digest"
        print(url)
        ## ADDING IN SELENIUM

        options = Options()
        options.add_argument('--disable-gpu')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')

        # Since there is no chromedriver.exe, Selenium should auto-detect the integrated driver
        driver = webdriver.Chrome(options=options)
        
        page_content = driver.page_source
        tree = html.fromstring(page_content)

        # get text for heading 1s
        check_valid = str(tree.xpath("//*//h1//text()"))
        
        driver.quit()

         ## ADDING IN SELENIUM
        
        while "We couldn't find that page" in check_valid:
            datestart = datestart + timedelta(days=1)
            url = "https://www.congress.gov/congressional-record/"+str('{d.year}/{d.month}/{d.day}'.format(d=datestart))+"/daily-digest"
            if datestart >= dateend:
                break
                
        return(TestClass(datestart,url))


  ### get strg ####

    #function to extract relevant info from datestring
    def to_append(str_chamber,url,chamber,datestart,cols):

        timein, adj, conv, strg = None, None, None, None
        if "not in session" in str_chamber or "sine die" in str_chamber:
            return(pd.DataFrame([[chamber,datestart,None,None,"NS",url]],columns=cols))
        elif "pro forma" in str_chamber:
            return(pd.DataFrame([[chamber,datestart,None,None,"PF",url]],columns=cols))

        else:
            times = list()

            for i in re.finditer(r'([0-1]?[0-9]|2[0-3]):?[0-5]?[0-9]? ?([AaPp][.]?[Mm][.]?)', str_chamber):
                times += (i.group())

            if len(times) < 2:

                times = list()

                for i in re.finditer(r'([0-1]?[0-9]|2[0-3]):?[0-5]?[0-9]? ?([AaPp][.]?[Mm][.]?)?', str_chamber):
                    times += (i.group())

                for time in range(len(times)):
                    if 'm' not in times[time]:
                        try:
                            for x in re.finditer(r'([AaPp][.]?[Mm][.]?)', str(times[0])):
                                pm_am = x.group()
                            times[time] = (times[time] + " " + pm_am)
                        except:
                            times[time] = (times[time] + " pm")

            try:
                times = [time.replace(".","") for time in times]
                conv = timeconv(times[0])
                adj = timeconv(times[1])
                timein = (adj-conv).seconds//60

                if timein >= 60:
                    return(pd.DataFrame([[chamber,datestart,conv.time(),adj.time(),"X",url]],columns=cols))
                else:
                    return(pd.DataFrame([[chamber,datestart,conv.time(),adj.time(),"PF",url]],columns=cols))

            except:
                if "and is in recess" in str_chamber or "stands in recess" in str_chamber or "still in session" in str_chamber or "stands adjourned" in str_chamber and times[0]:
                    return(pd.DataFrame([[chamber,datestart,timeconv(times[0]).time(),"-","X",url]],columns=cols))
                else:
                    return(pd.DataFrame([[chamber,datestart,"HOLD","HOLD","HOLD",url]],columns=cols))

    def errorcatch(url):
        
        ## ADDING IN SELENIUM

        options = Options()
        options.add_argument('--disable-gpu')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')

        # Since there is no chromedriver.exe, Selenium should auto-detect the integrated driver
        driver = webdriver.Chrome(options=options)

        webpage = driver.get(url)

        driver.quit()

        ## ADDING IN SELENIUM
        
        tree = html.fromstring(webpage)

        text = tree.xpath("//div[@class='txt-box']//text()")
        keywords = ["met at","adjourned","not in session","pro forma session"]

        if not text:
            text = tree.xpath("//div[@class='main-wrapper']//text()")

        ch = "House" if chamber == "House of Representatives" else "Senate"
        str_chamber = None

        for x in text:
            if "Adjournment" in x:
                if ch in x:
                    str_chamber = clean(str(x))
        if not str_chamber:
            str_chamber = list()
            for x in text:
                for key in keywords:
                    if key in x and ch in x:
                        str_chamber += (x) if x not in str_chamber else None
            str_chamber = clean(str(str_chamber))
        if not str_chamber:
            l = tree.xpath("//div[@class='main-wrapper']//following-sibling::center[h2[contains(./text(),'"+ch+"')]]//following-sibling::p//text()")
            str_chamber = clean(join(",",[i for i in l if ch in i]))
        return(to_append(str_chamber,url,chamber,datestart,cols))

    #for each chamber for each relevant day, get the info and put it into the results dataframe
    for y in range(diff):
        datestart1 = datestart + (timedelta(days=1)*0 if y == 0 else timedelta(days=1))
        url, datestart = web_test(datestart1).url, web_test(datestart1).datestart
        
        if datestart >= dateend:
            break
            

        
        str_chamber = get_strg(url,chamber)

        if not str_chamber:
            
            ## ADDING IN SELENIUM

            options = Options()
            options.add_argument('--disable-gpu')
            options.add_argument('--no-sandbox')
            options.add_argument('--disable-dev-shm-usage')

            # Since there is no chromedriver.exe, Selenium should auto-detect the integrated driver
            driver = webdriver.Chrome(options=options)

            driver.get(url)

            # Parse the HTML content with lxml
            page_content = driver.page_source
            tree = html.fromstring(page_content)

            driver.quit()

            ## ADDING IN SELENIUM

            if "more than one issue for" in str(tree.xpath("//div[@class='main-wrapper']//h2[@class='alt']//text()")):
                    urls = tree.xpath("//div[@class='main-wrapper']//ul[@class='plain margin7']//li//@href")
                    for url in urls:
                        str_chamber = get_strg(url,chamber)
                        app = to_append(str_chamber,url,chamber,datestart,cols)
                        df = pd.concat([df, app])
            elif "No digest text" in clean(str(tree.xpath("//div[@class='main-wrapper']//p//text()"))):
                congress = clean(str(tree.xpath("//div[@class='featured']//h1//span[contains(text(),'Congress')]//text()")))
                app = pd.DataFrame([[chamber,datestart,"HOLD","INPUT BY HAND: example available pdfs at the following link","congress.gov"+tree.xpath("//p[@class='daily-digest-navigation']//a//@href")[0],congress]],columns=cols)
                df = pd.concat([df, app])
            else:
                try:
                    app = errorcatch(url)
                    df = pd.concat([df, app])
                except:
                    df = pd.concat([df, pd.DataFrame([[chamber,datestart,"HOLD","HOLD","UNKNOWN ERROR: INPUT BY HAND",url]],columns=cols)])
        else:
            app = to_append(str_chamber,url,chamber,datestart,cols)
            df = pd.concat([df, app])
        
    month_quarters = {1:1,2:1,3:1,4:2,5:2,6:2,7:3,8:3,9:3,10:4,11:4,12:4}
    df["Quarter"] = [month_quarters.get(d.month) for d in df["Date"]]
    display(df)
    return(df)

In [30]:
workingdays("House","08/01/2024","08/02/2024")

https://www.congress.gov/congressional-record/2024/8/1/daily-digest
https://www.congress.gov/congressional-record/2024/8/1/daily-digest


,House or Senate,Date,Time Convened,Time Adjourned,Working Day?,Link,Quarter
0,House,2024-08-01,HOLD,HOLD,UNKNOWN ERROR: INPUT BY HAND,https://www.congress.gov/congressional-record/...,3


,House or Senate,Date,Time Convened,Time Adjourned,Working Day?,Link,Quarter
0,House,2024-08-01,HOLD,HOLD,UNKNOWN ERROR: INPUT BY HAND,https://www.congress.gov/congressional-record/...,3
